## **Análisis exploratorio (EDA) inversión de Criptomonedas**

### Nuestro proyecto se fundamenta en un análisis destinado a **inversiones a mediano y largo plazo**. Por consiguiente, no emplearemos datos OHLC (Open, High, Low, Close) ya que nuestra finalidad no se enfoca en operaciones de trading, sino en inversiones basadas en otros indicadores clave de `rendimiento (KPI) y objetivos`. Utilizaremos gráficas distintas a las velas para cumplir con nuestros objetivos de inversión.

In [1]:
%pip install pycoingecko

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, -1, Finished, Available)


[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: /nfs4/pyenv-1801c0e2-cfa3-49cd-a83f-2d4a33da5dbc/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SQLContext
from pycoingecko import CoinGeckoAPI
from pyspark.sql.types import *

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 9, Finished, Available)

In [3]:
# Crear una cadena de conexión
connection_string = "jdbc:sqlserver://server-cistelsa.database.windows.net:1433;databaseName=data-crypto;user=cistelsa;password=Af670504e@"

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 10, Finished, Available)

In [4]:
# Conectarse a la base de datos
df_cryptocurrencies = spark.read.jdbc(connection_string, "cryptocurrencies")
df_bitcoin_halvings = spark.read.jdbc(connection_string, "bitcoin_halvings")
df_crypto_date = spark.read.jdbc(connection_string, "crypto_date")
df_crypto_dev = spark.read.jdbc(connection_string, "crypto_dev")
df_crypto_price = spark.read.jdbc(connection_string, "crypto_price")
df_crypto_social = spark.read.jdbc(connection_string, "crypto_social")

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 11, Finished, Available)

In [5]:
df_cryptocurrencies.show()

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 12, Finished, Available)

+------------+------+------------+--------------------+------------------+------------+
|          id|symbol|        name|               image|      total_supply|  max_supply|
+------------+------+------------+--------------------+------------------+------------+
|    arbitrum|   arb|    Arbitrum|https://assets.co...|            1.0E10| 10000000000|
|     bitcoin|   btc|     Bitcoin|https://assets.co...|             2.1E7|    21000000|
|boba-network|  boba|Boba Network|https://assets.co...|             5.0E8|   500000000|
|elrond-erd-2|  egld|  MultiversX|https://assets.co...|       2.5860072E7|    31415926|
|    ethereum|   eth|    Ethereum|https://assets.co...|1.20211284999274E8|        null|
|gmd-protocol|   gmd|         GMD|https://assets.co...|           80000.0|       80000|
|         okb|   okb|         OKB|https://assets.co...|     2.359576853E8|   300000000|
|    optimism|    op|    Optimism|https://assets.co...|     4.294967296E9|  4294967296|
|      ripple|   xrp|         XR

### Realizamos una exploración profunda para poder extraer el suministro circulante de criptomonedas `circulating_supply` en forma de histórico, pero no fué posible ya que las APIs no suministran esta información específica, solo en tiempo real, es una data muy importante para la toma de decisiones y revisar que sucedió en el pasado, pero no vamos a frenar el proyecto por este inconveniente, lo tomaremos en tiempo real a continuación, aun así en el README.md va a quedar estipulado que nuestro proyecto sigue avanzando y tendrá actualizaciones dentro de las tareas a desarrollar está:
- ### La automatización del proceso de extracción en tiempo real a término indefinido incluyendo `circulating_supply`
- ### Investigación y búsqueda de la data histórica de cada criptomóneda del proyecto con relación a `circulating_supply`

In [6]:
# Declaro la función de la api de coingecko en una variable cg
cg = CoinGeckoAPI()

# Lista de las critomonedas seleccionadas para análisis
crypto_list = ['arbitrum', 'bitcoin', 'boba-network', 'elrond-erd-2', 'ethereum', 'gmd-protocol', 'okb', 'optimism', 'ripple', 'solana' ]

coin_market = cg.get_coins_markets(vs_currency='usd', ids=crypto_list)

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 13, Finished, Available)

In [8]:
# Creamos primero el esquema que va a tener nuestro dataframe en este caso solo 2 columnas
orderSchema = StructType([
    StructField("id", StringType()),
    StructField("circulating_supply", FloatType())
    ])

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 15, Finished, Available)

In [9]:
# Crear un DataFrame de Spark a partir de los datos de CoinGecko
df_spark_cp = spark.createDataFrame(coin_market, schema=orderSchema)

# Agregamos la columna de los datos al dataframe df_cryptocurrencies por medio de un join
df_cryptocurrencies = df_cryptocurrencies.join(df_spark_cp.select("id", "circulating_supply"), on="id", how="left")

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 16, Finished, Available)

In [27]:
df_cryptocurrencies.show()

StatementMeta(, a6956720-8bc1-48f3-9206-8da34cd0ddad, 34, Finished, Available)

+------------+------+------------+--------------------+------------------+------------+------------------+
|          id|symbol|        name|               image|      total_supply|  max_supply|circulating_supply|
+------------+------+------------+--------------------+------------------+------------+------------------+
|      solana|   sol|      Solana|https://assets.co...|5.55286963547684E8|        null|      4.07933792E8|
|gmd-protocol|   gmd|         GMD|https://assets.co...|           80000.0|       80000|          59146.73|
|      ripple|   xrp|         XRP|https://assets.co...|   9.9988501123E10|100000000000|      5.2914192E10|
|boba-network|  boba|Boba Network|https://assets.co...|             5.0E8|   500000000|      3.42812544E8|
|     bitcoin|   btc|     Bitcoin|https://assets.co...|             2.1E7|    21000000|       1.9467524E7|
|    arbitrum|   arb|    Arbitrum|https://assets.co...|            1.0E10| 10000000000|      1.27500006E9|
|         okb|   okb|         OKB|htt

In [11]:
# Renombrar la columna en el DataFrame df_crypto_price
df_crypto_price = df_crypto_price.withColumnRenamed("cryptocurrency_id", "id")
# Agregamos la columna de los datos al dataframe df_crypto_price por medio de un join
df_crypto_price = df_crypto_price.join(df_cryptocurrencies.select("id", "max_supply", "circulating_supply"), on="id", how="left")

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 18, Finished, Available)

In [20]:
# Hacemos otro Join para agregarle la fecha de la data
df_crypto_price = df_crypto_price.join(df_crypto_date.select("date_id", "date"), on="date_id", how="left")

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 29, Finished, Available)

In [21]:
new_crypto_price = df_crypto_price.select("id", "date", "price_usd", "market_cap_usd", "total_volume_usd", "max_supply", "circulating_supply")

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 30, Finished, Available)

In [1]:
from pyspark.sql.functions import col

# Supongamos que tienes un DataFrame new_crypto_price
new_crypto_price = new_crypto_price \
    .withColumn("price_usd", col("price_usd").cast("float")) \
    .withColumn("market_cap_usd", col("market_cap_usd").cast("int")) \
    .withColumn("total_volume_usd", col("total_volume_usd").cast("int")) \
    .withColumn("max_supply", col("max_supply").cast("int")) \
    .withColumn("circulating_supply", col("circulating_supply").cast("int")) \
    .withColumn("date", col("date").cast("date"))

StatementMeta(, 0758cb9f-00ca-4f54-b647-acdc383726cb, 3, Submitted, Waiting)

In [26]:
new_crypto_price.describe()

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 35, Finished, Available)

DataFrame[summary: string, id: string, price_usd: string, market_cap_usd: string, total_volume_usd: string, max_supply: string, circulating_supply: string]

### Es necesario tener el dato de la capitalización de mercado máxima de cada cryptomoneda y tener una referencia para tomar alguna decisión en un ciclo alsista este puede superarse multiplicado x

In [25]:
marketCapMax = new_crypto_price.select("id", "date", "market_cap_usd").groupBy("id").agg(max("market_cap_usd").alias("MaximoMarket"))
display(marketCapMax)

StatementMeta(, e220d1f8-cb1a-45ad-bf32-fbc7e99908d6, 34, Finished, Available)

AttributeError: 'str' object has no attribute 'alias'